In [17]:
import os
import io
import csv
import pandas as pd
import numpy as np

import psycopg2
conn = psycopg2.connect(user='airflow', password='Xypherium-0',
                        dbname='jpstat',
                        host='35.224.240.50')

In [2]:
pd.set_option('display.max_colwidth', -1)

In [3]:
path = os.path.join('extracts', 'monthly_reports')

In [4]:
def get_directory_structure(rootdir):
    """
    Creates a nested dictionary that represents the folder structure of rootdir
    """
    dir = {}
    rootdir = rootdir.rstrip(os.sep)
    start = rootdir.rfind(os.sep) + 1
    for path, dirs, files in os.walk(rootdir):
        folders = path[start:].split(os.sep)
        subdir = dict.fromkeys(files)
        parent = reduce(dict.get, folders[:-1], dir)
        parent[folders[-1]] = subdir
    return dir

In [5]:
filepathlist = []
for dirpath, subdir, files in os.walk(path):
    for filename in files:
        if filename.endswith('.xls'):
            filepathlist.append(os.path.join(dirpath, filename))

In [6]:
%%time
tableshape_list = []
df_dict = {}
for filepath in filepathlist:
    sheets = pd.ExcelFile(filepath).sheet_names
    
    df = pd.read_excel(filepath, sheet_name=len(sheets)-1)
    filepath_list = filepath.split('\\')
    description = [filepath_list[2] + '-' + filepath_list[3], filepath_list[4], df.shape]
    
    if len(sheets) > 1:
        print(filepath_list[2] + '-' + filepath_list[3] + ' ' + filepath_list[4] + ' has ' + str(len(sheets)) + ' sheets.')
    
    tableshape_list.append(description)

2013-07Jul 1.xls has 2 sheets.
2013-07Jul 3-3.xls has 2 sheets.
2013-08Aug 1.xls has 2 sheets.
2013-08Aug 3-3.xls has 2 sheets.
2013-09Sep 1.xls has 2 sheets.
2013-09Sep 3-3.xls has 2 sheets.
2013-10Oct 1.xls has 2 sheets.
2013-10Oct 3-3.xls has 2 sheets.
2013-11Nov 1.xls has 2 sheets.
2013-11Nov 3-3.xls has 2 sheets.
2013-12Dec 1.xls has 2 sheets.
2013-12Dec 3-3.xls has 2 sheets.
2014-01Jan 1.xls has 2 sheets.
2014-01Jan 3-3.xls has 2 sheets.
2014-02Feb 1.xls has 2 sheets.
2014-02Feb 3-3.xls has 2 sheets.
2014-03Mar 1.xls has 2 sheets.
2014-03Mar 3-3.xls has 2 sheets.
2014-04Apr 1.xls has 2 sheets.
2014-04Apr 3-3.xls has 2 sheets.
2014-05May 1.xls has 2 sheets.
2014-05May 3-3.xls has 2 sheets.
2014-06Jun 1.xls has 2 sheets.
2014-06Jun 3-3.xls has 2 sheets.
2014-07Jul 1.xls has 2 sheets.
2014-07Jul 3-3.xls has 2 sheets.
2014-08Aug 1.xls has 2 sheets.
2014-08Aug 3-3.xls has 2 sheets.
2014-09Sep 1.xls has 2 sheets.
2014-09Sep 3-3.xls has 2 sheets.
2014-10Oct 1.xls has 2 sheets.
2014-10Oc

In [7]:
tableshape_df = pd.DataFrame(tableshape_list, columns=['year-month', 'file', 'shape'])

In [8]:
tableshape_agg_df = tableshape_df.groupby(['file', 'shape'])['year-month'].apply(list).reset_index()

In [9]:
tableshape_agg_df['count'] = [len(x) for x in tableshape_agg_df['year-month']]
[x.sort for x in tableshape_agg_df['year-month']]
tableshape_agg_df.sort_values(by='year-month')
tableshape_agg_df

,file,shape,year-month,count
0,1.xls,"(86, 18)","[2006-10Oct, 2006-11Nov, 2006-12Dec, 2007-01Jan, 2007-02Feb, 2007-03Mar]",6
1,1.xls,"(88, 13)","[2006-01Jan, 2006-02Feb, 2006-03Mar]",3
2,1.xls,"(88, 18)","[2007-04Apr, 2007-05May, 2007-06Jun, 2007-07Jul, 2007-08Aug, 2007-09Sep, 2007-10Oct, 2007-11Nov, 2007-12Dec]",9
3,1.xls,"(89, 13)","[2006-04Apr, 2006-05May, 2006-06Jun, 2006-07Jul, 2006-08Aug, 2006-09Sep]",6
4,1.xls,"(91, 13)","[2008-01Jan, 2008-02Feb, 2008-03Mar, 2008-04Apr, 2008-05May, 2008-06Jun, 2008-07Jul, 2008-08Aug, 2008-09Sep, 2008-10Oct, 2008-11Nov, 2008-12Dec, 2009-01Jan, 2009-02Feb, 2009-03Mar, 2010-01Jan, 2010-02Feb, 2010-03Mar]",18
5,1.xls,"(92, 13)","[2009-04Apr, 2009-05May, 2009-06Jun, 2009-07Jul, 2009-08Aug, 2009-09Sep, 2009-10Oct, 2009-11Nov, 2009-12Dec, 2010-04Apr, 2010-05May, 2010-06Jun, 2010-07Jul, 2010-08Aug, 2010-09Sep, 2010-10Oct, 2010-11Nov, 2010-12Dec, 2011-01Jan, 2011-02Feb, 2011-03Mar, 2011-04Apr, 2011-05May, 2011-06Jun, 2011-07Jul, 2011-08Aug, 2011-09Sep, 2011-10Oct, 2011-11Nov, 2011-12Dec, 2012-01Jan, 2012-02Feb, 2012-03Mar]",33
6,1.xls,"(93, 13)","[2012-04Apr, 2012-05May, 2012-06Jun, 2012-07Jul, 2012-08Aug, 2012-09Sep, 2012-10Oct, 2012-11Nov, 2012-12Dec, 2013-01Jan, 2013-02Feb, 2013-03Mar, 2013-04Apr, 2013-05May, 2013-06Jun, 2013-07Jul, 2013-08Aug, 2013-09Sep, 2013-10Oct, 2013-11Nov, 2013-12Dec, 2014-01Jan, 2014-02Feb, 2014-03Mar, 2014-04Apr, 2014-05May, 2014-06Jun, 2014-07Jul, 2014-08Aug, 2014-09Sep, 2014-10Oct, 2014-11Nov, 2014-12Dec, 2015-01Jan, 2015-02Feb, 2015-03Mar, 2015-04Apr, 2015-05May, 2015-06Jun, 2015-07Jul, 2015-08Aug, 2015-09Sep, 2015-10Oct, 2015-11Nov, 2015-12Dec, 2016-01Jan, 2016-02Feb, 2016-03Mar, 2016-04Apr, 2016-05May, 2016-06Jun, 2016-07Jul, 2016-08Aug, 2016-09Sep, 2016-10Oct, 2016-11Nov, 2016-12Dec, 2017-01Jan, 2017-02Feb, 2017-03Mar, 2017-04Apr, 2017-05May, 2017-06Jun, 2017-07Jul, 2017-08Aug, 2017-09Sep, 2017-10Oct, 2017-11Nov, 2017-12Dec]",69
7,1.xls,"(105, 13)","[2018-01Jan, 2018-02Feb, 2018-03Mar, 2018-04Apr, 2018-05May, 2018-06Jun]",6
8,2.xls,"(1285, 12)","[2006-01Jan, 2006-02Feb, 2006-03Mar]",3
9,2.xls,"(1362, 18)","[2006-10Oct, 2006-11Nov, 2006-12Dec, 2007-01Jan, 2007-02Feb, 2007-03Mar]",6


In [10]:
table3_1_list = [x for x in filepathlist if '3-1' in x]
table3_2_list = [x for x in filepathlist if '3-2' in x]

In [11]:
%%time
#Always take the last sheet: assume that in the absense of separate sheets, the numbers are for Japanese citizens only
table_3_1_df_list = []

for filepath in table3_1_list:
    filepath_parts = filepath.split('\\')
    year = filepath_parts[2]
    month = filepath_parts[3][:2]
    
    sheets = pd.ExcelFile(filepath).sheet_names
    df = pd.read_excel(filepath, sheet_name=len(sheets)-1)
    
    messy_df = df.copy()
    messy_df.reset_index(inplace=True)
    messy_df.columns = [x.replace('～', '-') for x in messy_df.iloc[13].replace({
                                                'Prefectures': 'Prefecture Num',
                                                '総 数 1)': 'Total',
                                                '0～4歳': '0～4',
                                                '90歳以上': '90 & above'
                                                }).fillna('Prefecture')]
    messy_df.iloc[18,10] = messy_df.iloc[18,8]
    messy_df = messy_df.iloc[18:66,8:]
    messy_df.reset_index(drop=True, inplace=True)
    messy_df.dropna(axis=1, inplace=True)
    messy_df.loc[0,'Prefecture Num'] = '00'
    
    both_df = messy_df.iloc[:,:22]
    male_df = messy_df.iloc[:,22:44]
    female_df = messy_df.iloc[:,44:66]
    
    both_df['Gender'] = 'Both'
    male_df['Gender'] = 'Male'
    female_df['Gender'] = 'Female'
    
    this_month_df = pd.concat([both_df, male_df, female_df])
    this_month_df['Year'] = str(year)
    this_month_df['Month'] = str(month)
    
    if False not in this_month_df.iloc[:,2:22].applymap(np.isreal).values.flatten():
        table_3_1_df_list.append(this_month_df)
    else:
        print('Some non-numeric value found in counts.')

C:\Users\friedemann.ang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\friedemann.ang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\friedemann.ang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

Wall time: 11.1 s


In [12]:
in_migrants_age_df = pd.concat(table_3_1_df_list)

In [14]:
in_migrants_age_df

,Prefecture Num,Prefecture,Total,0-4,5-9,10-14,15-19,20-24,25-29,30-34,...,60-64,65-69,70-74,75-79,80-84,85-89,90 & above,Gender,Year,Month
0,00,全国,126018,7924,3182,1637,3610,20043,26958,20002,...,3247,1722,1098,946,864,711,387,Both,2010,01
1,01,北海道,2288,139,75,43,78,361,477,334,...,52,23,9,8,5,4,3,Both,2010,01
2,02,青森県,954,72,32,20,66,182,175,107,...,19,7,2,6,2,2,1,Both,2010,01
3,03,岩手県,866,67,36,14,24,155,163,123,...,20,10,8,3,2,2,2,Both,2010,01
4,04,宮城県,2229,178,70,29,54,412,416,350,...,52,25,12,6,8,13,6,Both,2010,01
5,05,秋田県,666,46,35,5,22,120,120,112,...,20,11,0,2,1,4,0,Both,2010,01
6,06,山形県,646,68,21,2,20,137,137,86,...,12,6,3,3,2,1,1,Both,2010,01
7,07,福島県,1318,83,46,26,35,232,244,192,...,50,21,14,11,8,6,2,Both,2010,01
8,08,茨城県,2582,184,69,28,56,398,536,379,...,75,48,41,32,26,17,12,Both,2010,01
9,09,栃木県,1949,147,56,23,51,316,367,304,...,59,21,12,15,13,10,7,Both,2010,01


In [18]:
cur = conn.cursor()
in_migrants_textstream = io.StringIO()
upload_df = in_migrants_age_df.copy()
#upload_df['excel_description'].replace(['\n', '\t'], '', regex=True, inplace=True)

upload_df.to_csv(in_migrants_textstream, sep='\t', header=False, index=False, quoting=csv.QUOTE_NONE)
in_migrants_textstream.seek(0) 
cur.copy_from(in_migrants_textstream, 'jpstat_inmigrants', null="") # null values become ''
conn.commit()

In [ ]:
#write using pyarrow with flavor=spark so that the files are compatible (datatypes)
